In [1]:
import gym
import numpy as np
from collections import defaultdict

In [2]:
env = gym.make('Blackjack-v1')

C:\Users\triet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
C:\Users\triet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
# Initialize Q-values, returns, and policy
Q = defaultdict(lambda: np.zeros(env.action_space.n))  # Q-value for each state-action pair
returns_sum = defaultdict(float)  # Total return for each state-action pair
returns_count = defaultdict(float)  # Count of how many times each state-action pair is visited

In [4]:
# Hyperparameters
epsilon = 0.1  # Exploration rate
gamma = 1.0    # Discount factor

In [5]:
# Define epsilon-greedy policy
def epsilon_greedy_policy(state, epsilon=0.1):
    if np.random.rand() < epsilon:
        return env.action_space.sample()  # Explore
    else:
        return np.argmax(Q[state])  # Exploit

In [6]:
# Generate episode
def generate_episode(policy, epsilon):
    episode = []
    state = env.reset()  # Older gym versions return only state, no info
    done = False
    
    while not done:
        action = policy(state, epsilon)  # Choose action using ε-greedy policy
        next_state, reward, done, info = env.step(action)  # Older gym returns 4 values from step()
        episode.append((state, action, reward))
        state = next_state
    
    return episode

In [7]:
# Update Q-values based on the episode
def update_Q(episode, gamma):
    G = 0  # Initialize the return
    visited_state_action_pairs = set()  # Keep track of visited state-action pairs to avoid duplicates
    
    # Work backwards through the episode
    for state, action, reward in reversed(episode):
        G = gamma * G + reward  # Calculate the cumulative return
        
        if (state, action) not in visited_state_action_pairs:
            returns_sum[(state, action)] += G
            returns_count[(state, action)] += 1
            Q[state][action] = returns_sum[(state, action)] / returns_count[(state, action)]  # Update Q-value
            visited_state_action_pairs.add((state, action))

In [8]:
# Train the agent using Monte Carlo control
def train_blackjack_MC(episodes, epsilon=0.1, gamma=1.0):
    for i in range(episodes):
        # Generate an episode following the ε-greedy policy
        episode = generate_episode(epsilon_greedy_policy, epsilon)
        
        # Update Q-values using the generated episode
        update_Q(episode, gamma)
        
    print("Training completed!")

In [9]:
# Test the agent's performance
def test_blackjack_MC(episodes):
    wins = 0
    losses = 0
    draws = 0

    for _ in range(episodes):
        state = env.reset()  # Older gym returns only state
        done = False

        while not done:
            action = np.argmax(Q[state])  # Always exploit the learned Q-values
            state, reward, done, info = env.step(action)  # Older gym returns 4 values

        if reward > 0:
            wins += 1
        elif reward < 0:
            losses += 1
        else:
            draws += 1

    print(f"Results after {episodes} episodes: {wins} wins, {losses} losses, {draws} draws")
    print('Win rate:' , (wins / episodes) * 100 , '%')

In [10]:
# Train the agent
train_blackjack_MC(5000)  # Train for 500,000 episodes

# Test the agent
test_blackjack_MC(100)  # Test for 1,000 episodes

C:\Users\triet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gym\utils\passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Training completed!
Results after 100 episodes: 37 wins, 58 losses, 5 draws
Win rate: 37.0 %
